In [1]:
import keras
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32') / 255.
y_train = keras.utils.to_categorical(y_train)

x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test = x_test.astype('float32') / 255.
y_test = keras.utils.to_categorical(y_test)

In [2]:
from keras.models import Model
from keras.layers import Flatten, Conv2D, Conv2DTranspose, Dense, Input, Reshape, concatenate, Activation
from keras.utils import plot_model

# encoder
enc_input = Input(shape=(28,28,1), name='enc_input')
x  = Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), activation='relu', padding='same', name='conv1')(enc_input)
x  = Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), activation='relu', padding='same', name='conv2')(x)
x  = Conv2D(filters=1, kernel_size=(3,3), strides=(2,2), activation='relu', padding='same', name='pt_conv1')(x)
x = Flatten(name='flatten')(x)
latent = Activation('relu', name='latent')(x)

# decoder
dec_input = Input(shape=(16,), name='dec_input')
x = Dense(units=7*7*8, activation='relu', name='dense')(dec_input)
x = Reshape(target_shape=(7,7,8), name='reshape')(x)
x = Conv2DTranspose(filters=64, kernel_size=3, strides=2, activation='relu', padding='same', name='deconv1')(x)
x = Conv2DTranspose(filters=32, kernel_size=3, strides=2, activation='relu', padding='same', name='deconv2')(x)
dec_output = Conv2DTranspose(filters=1, kernel_size=3, padding='same', name='pt_conv')(x)

encoder = Model(enc_input, latent, name="Encoder")
decoder = Model(dec_input, dec_output, name="Decoder")
model = Model(encoder.input, decoder(encoder.output))

model.compile(loss='mse', optimizer='adam')
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_input (InputLayer)       [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 14, 14, 32)        320       
_________________________________________________________________
conv2 (Conv2D)               (None, 7, 7, 64)          18496     
_________________________________________________________________
pt_conv1 (Conv2D)            (None, 4, 4, 1)           577       
_________________________________________________________________
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
latent (Activation)          (None, 16)                0         
_________________________________________________________________
Decoder (Model)              (None, 28, 28, 1)         30089 

In [3]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('../models/49_1split.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

history = model.fit(x_train, x_train, validation_data=(x_test, x_test), epochs=100, batch_size=32, callbacks=[es, mc])

Epoch 1/100
1871/1875 [============================>.] - ETA: 0s - loss: 0.0295
Epoch 00001: val_loss improved from inf to 0.02332, saving model to ../models/49_1split.h5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0295 - val_loss: 0.0233
Epoch 2/100
1865/1875 [============================>.] - ETA: 0s - loss: 0.0213
Epoch 00002: val_loss improved from 0.02332 to 0.01998, saving model to ../models/49_1split.h5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0213 - val_loss: 0.0200
Epoch 3/100
1862/1875 [============================>.] - ETA: 0s - loss: 0.0197
Epoch 00003: val_loss improved from 0.01998 to 0.01907, saving model to ../models/49_1split.h5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0197 - val_loss: 0.0191
Epoch 4/100
1873/1875 [============================>.] - ETA: 0s - loss: 0.0189
Epoch 00004: val_loss improved from 0.01907 to 0.01819, saving model to ../models/49_1split.h5
1875/1875 [====================

KeyboardInterrupt: 

In [2]:
import sys; sys.path.insert(0, '..')
import utils
import keras
import numpy as np

def embed_and_translate(data, n_width, n_height):
    ndata = np.zeros((len(data), n_width, n_height, 1), dtype='float32')
    translations = np.empty((len(data), 2), dtype='float32')
    width, height = data.shape[1], data.shape[2]
    for i in range(len(data)):
        x = np.random.randint(n_width-width)
        y = np.random.randint(n_height-height)
        ndata[i][x:x+width, y:y+height] = data[i] # rows, cols = height, width
        translations[i][0] = x+(width//2)
        translations[i][1] = y+(height//2)
    return ndata, translations
            
n_splits = 16
output_shape = (56, 56, 1)
input_shape  = (14, 14, 1)
split_x, split_y = 14, 14
latent_dim = 4

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32') / 255.
x_train_augmented, y_train_regr = embed_and_translate(x_train, output_shape[0], output_shape[1])
x_train_split = np.array([utils.split(x, split_x, split_y) for x in x_train_augmented], dtype='float32')
y_train = keras.utils.to_categorical(y_train)

x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test = x_test.astype('float32') / 255.
x_test_augmented, y_test_regr = embed_and_translate(x_test, output_shape[0], output_shape[1])
x_test_split = np.array([utils.split(x, split_x, split_y) for x in x_test_augmented], dtype='float32')
y_test = keras.utils.to_categorical(y_test)

In [3]:
from keras.layers import Flatten, Conv2D, Conv2DTranspose, Dense, Input, Reshape, concatenate, Activation, Dropout, MaxPooling2D
from keras import Sequential
from keras.models import Model

import tensorflow as tf
import keras
import sys; sys.path.insert(0, '..')
import utils

class MultiSplit(Model):
    def __init__(self, n_splits, latent_dim, input_shape, output_shape):
        super(MultiSplit, self).__init__()
        self.encoder = self._create_encoder(latent_dim, input_shape)
        self.input_reshaper = utils.Reshaper((n_splits, *input_shape), input_shape)
        self.latent_reshaper = utils.Reshaper([latent_dim], [n_splits * latent_dim])
        self.decoder = self._create_decoder(n_splits * latent_dim, output_shape)
        self.classifier = self._create_classifier(n_splits * latent_dim)
        self.regressor = self._create_regressor(n_splits * latent_dim)
        
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x)
            loss = self.compiled_loss(y, y_pred)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        x, y = data
        y_pred = self(x, training=False)
        y_pred['decoder_out'] = tf.image.extract_glimpse(y_pred['decoder_out'], (28,28), y['regressor_out'], centered=False, normalized=False, noise='zero')
        self.compiled_loss(y, y_pred)
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def call(self, x, training=True):
        z = self.encode(x, training=training)
        y_reco = self.decode(z ,training=training)
        y_class = self.classify(z ,training=training)
        y_regr = self.regress(z ,training=training)
        return {"decoder_out": y_reco, "classifier_out": y_class, "regressor_out": y_regr}
        
    def encode(self, x, training=True):
        return self.encoder( self.input_reshaper(x) , training)
    
    def decode(self, z, training=True):
        return self.decoder( self.latent_reshaper(z) , training)
    
    def classify(self, z, training=True):
        return self.classifier( self.latent_reshaper(z) , training)
    
    def regress(self, z, training=True):
        return self.regressor( self.latent_reshaper(z) , training)
    
    def _create_encoder(self, latent_dim, input_shape, n_filters=[32,64]):
        return Sequential([
            Input(shape=input_shape),
            Conv2D(filters=n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2D(filters=n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2D(filters=1, kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Flatten(),
            Activation('sigmoid')
#             Dense(units=latent_dim, activation='sigmoid')
        ], name='encoder')

    def _create_decoder(self, latent_dim, io_shape, n_filters=[32,64]):
        return Sequential([
            Input(shape=(latent_dim)),
            Dense(io_shape[0]//4 * io_shape[1]//4 * 8),  #! Reduce amount of neurons by 4.
            Reshape((io_shape[0]//4, io_shape[1]//4, 8)),
            Conv2DTranspose(filters=n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2DTranspose(filters=n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2DTranspose(filters=1, kernel_size=(3,3), padding='same'),
            Activation('sigmoid', name='decoder_out')
        ])
    
    def _create_classifier(self, latent_dim):
        return Sequential([
            Input(shape=(latent_dim)),
            Dense(256, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(10, activation='softmax', name='classifier_out')
        ])
    
    def _create_regressor(self, latent_dim):
        return Sequential([
            Input(shape=(latent_dim)),
            Dense(256, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(10, activation='relu'),
            Dense(2, activation='linear', name='regressor_out')
        ])

In [4]:
losses = {
    'decoder_out' : 'mse',
    'classifier_out' : 'categorical_crossentropy',
    'regressor_out' : 'mean_absolute_percentage_error'
}
loss_weights = {
    'decoder_out' : 0.0,
    'classifier_out' : 0.0,
    'regressor_out' : 1.0
}
model = MultiSplit(n_splits, latent_dim, input_shape, output_shape)
model.compile(loss=losses, loss_weights=loss_weights, optimizer='adam')

In [5]:
history = model.fit(x_train_split, {'decoder_out': x_train_augmented, 'classifier_out': y_train, 'regressor_out': y_train_regr},
                    validation_data=(x_test_split, {'decoder_out': x_test, 'classifier_out': y_test, 'regressor_out':y_test_regr}),
                    epochs=25, batch_size=32)

Epoch 1/25
1875/1875 [==============================] - 25s 13ms/step - loss: 9.1226 - classifier_out_loss: 2.3198 - decoder_out_loss: 0.2462 - regressor_out_loss: 9.1226 - val_loss: 3.5514 - val_classifier_out_loss: 2.3179 - val_decoder_out_loss: 0.2321 - val_regressor_out_loss: 3.5514
Epoch 2/25
1875/1875 [==============================] - 25s 13ms/step - loss: 3.4706 - classifier_out_loss: 2.3182 - decoder_out_loss: 0.2462 - regressor_out_loss: 3.4706 - val_loss: 2.9008 - val_classifier_out_loss: 2.3176 - val_decoder_out_loss: 0.2321 - val_regressor_out_loss: 2.9008
Epoch 3/25
1875/1875 [==============================] - 25s 13ms/step - loss: 2.9552 - classifier_out_loss: 2.3179 - decoder_out_loss: 0.2461 - regressor_out_loss: 2.9552 - val_loss: 2.7744 - val_classifier_out_loss: 2.3171 - val_decoder_out_loss: 0.2321 - val_regressor_out_loss: 2.7744
Epoch 4/25
1875/1875 [==============================] - 25s 13ms/step - loss: 2.5558 - classifier_out_loss: 2.3178 - decoder_out_loss: 0